In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from autoddg.related.related import RelatedWorkProfiler

# Load environment variables
load_dotenv()

# Your existing MODEL_CONFIG
MODEL_CONFIG = {
    "base_url": "https://openrouter.ai/api/v1",
    "api_key": os.getenv("OPENROUTER_API_KEY"), 
    "model_name": "mistralai/mistral-7b-instruct:free", 
}

# Create client
client = OpenAI(
    api_key=MODEL_CONFIG["api_key"],
    base_url=MODEL_CONFIG["base_url"]
)

# Create profiler
profiler = RelatedWorkProfiler(
    client=client,
    model_name=MODEL_CONFIG["model_name"]
)

In [5]:
# Test it!
profile = profiler.analyze_paper(
    pdf_path="../src/autoddg/related/papers/code15.pdf",
    dataset_name="CODE-15%: a large scale annotated dataset of 12-lead ECGs",
    max_pages=10  # Limit pages for quick testing
)

# See results
print(profile['summary'])

Ignoring wrong pointing object 43 0 (offset 0)


Reading PDF from: ../src/autoddg/related/papers/code15.pdf
Successfully extracted text from 10 pages (total: 10 pages)
Total characters extracted: 55673
Extracting related work profile for dataset: CODE-15%: a large scale annotated dataset of 12-lead ECGs
Sending 56312 characters to LLM...
Successfully extracted profile (1968 characters)
CODE-15% is a large-scale annotated dataset of 12-lead ECGs, derived from the Telehealth Network of Minas Gerais (TNMG) dataset. It contains 7–10-second-long recordings resampled to 400 Hz, with a focus on cardiac arrhythmia diagnosis, atrial fibrillation (AF) risk prediction, and age estimation. The dataset includes 6 arrhythmia classes (first-degree atrioventricular block, right and left bundle branch block, sinus bradycardia, AF, and sinus tachycardia) and normal sinus rhythm (NSR) as a control. For AF risk prediction, recordings were classified as "Future AF" (patients with subsequent AF diagnosis within 5 years) or "Non-AF" (patients without AF in